In [ ]:
!ls Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
#import the four relevent schemas to the budget and genres
tndb_df = pd.read_csv('Data/tn.movie_budgets.csv')
tn_mov_df = pd.read_csv('Data/tmdb.movies.csv')
akas_df = pd.read_csv('Data/imdb.title.akas.csv')
basics_df = pd.read_csv('Data/imdb.title.basics.csv')


In [ ]:
#display(tn_mov_df.shape)
#tn_mov_df['popularity'].unique
#no idea what how they measure popularity so this data will not be used

In [3]:
#removed all punctuations to movie titles on the two schemas
tndb_df["Mov_name"] = tndb_df['movie'].str.lower().str.replace('[ \'\:\,\.]','').astype(str)
tn_mov_df["Mov_name"] = tn_mov_df['title'].str.lower().str.replace('[ \'\:\,\.]','').astype(str)

#display(basics_df.head(1))
#display(akas_df.head(1))

In [7]:
#merged the two schemas with constant id
#rt_df = info_df.merge(reviews_df, on='id')

imdb_df = basics_df.merge(akas_df,left_on ='tconst',right_on = "title_id",how = "inner") 

#merged tn and tndb to create new db, keeping values only the same movie names 
tn_df = tn_mov_df.merge(tndb_df, on = 'Mov_name', how = 'inner')

In [ ]:
#imdb_df

In [8]:
display(imdb_df.head(1))
tn_df.head(1)

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,title_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",tt0063540,1,Sangharsh,IN,hi,NaN,alternative transliteration,0.0


,Unnamed: 0,genre_ids,id_x,original_language,original_title,popularity,release_date_x,title,vote_average,vote_count,Mov_name,id_y,release_date_y,movie,production_budget,domestic_gross,worldwide_gross
0,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,howtotrainyourdragon,30,"Mar 26, 2010",How to Train Your Dragon,"$165,000,000","$217,581,232","$494,870,992"


In [9]:
#created new column with movie names stripped
imdb_df['Mov_name'] = imdb_df['primary_title'].str.lower().str.replace('[ \'\:\,\.]','').astype(str)

In [12]:
imdb_df.head(10)

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,title_id,ordering,title,region,language,types,attributes,is_original_title,Mov_name
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",tt0063540,1,Sangharsh,IN,hi,NaN,alternative transliteration,0.0,sunghursh
1,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",tt0063540,2,Sunghursh,NaN,NaN,original,NaN,1.0,sunghursh
2,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",tt0063540,3,Sunghursh,IN,NaN,NaN,NaN,0.0,sunghursh
3,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",tt0063540,4,Sunghursh,IN,hi,NaN,alternative transliteration,0.0,sunghursh
4,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",tt0063540,5,Sungharsh,IN,hi,NaN,alternative spelling,0.0,sunghursh
5,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",tt0066787,1,One Day Before the Rainy Season,XWW,en,NaN,NaN,0.0,onedaybeforetherainyseason
6,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",tt0066787,2,Ashad Ka Ek Din,IN,NaN,NaN,NaN,0.0,onedaybeforetherainyseason
7,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",tt0066787,3,One Monsoon Day,NaN,NaN,NaN,NaN,0.0,onedaybeforetherainyseason
8,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",tt0066787,4,Ashad Ka Ek Din,NaN,NaN,original,NaN,1.0,onedaybeforetherainyseason
9,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,tt0069049,10,La otra cara del viento,VE,NaN,NaN,NaN,0.0,theothersideofthewind


In [13]:
#new cost df joining imdb and tn keeping only values with the same name and has cost data
p_cost = tn_df.merge(imdb_df, on = 'Mov_name', how = 'inner')

In [23]:
#p_cost.columns

Index(['Unnamed: 0', 'genre_ids', 'id_x', 'original_language',
       'original_title_x', 'popularity', 'release_date_x', 'title_x',
       'vote_average', 'vote_count', 'Mov_name', 'id_y', 'release_date_y',
       'movie', 'production_budget', 'domestic_gross', 'worldwide_gross',
       'tconst', 'primary_title', 'original_title_y', 'start_year',
       'runtime_minutes', 'genres', 'title_id', 'ordering', 'title_y',
       'region', 'language', 'types', 'attributes', 'is_original_title'],
      dtype='object')

In [25]:
#p_cost[['original_title_x','title_x', 'Mov_name', 'movie', 'primary_title','original_title_y','title_y']]

In [26]:
#created two new columns to convert money strings to ints without punctuations
p_cost["domestic_gross_$"] = p_cost['domestic_gross'].str.lower().str.replace('[ \'\:\,\.\$]','').astype(int)
p_cost["worldwide_gross_$"] = p_cost['worldwide_gross'].str.lower().str.replace('[ \'\:\,\.\$]','').astype(int)

In [27]:
#new column converting production budget into a usable format
p_cost["production_$"] = p_cost['production_budget'].str.lower().str.replace('[ \'\:\,\.\$]','').astype(int)

#new column with the sum of the gross
p_cost['total_gross'] = p_cost["worldwide_gross_$"] + p_cost["domestic_gross_$"]

#new colum with net value of the movie
p_cost['net_cost'] = p_cost['total_gross'] - p_cost['production_$']


display(p_cost.columns)


Index(['Unnamed: 0', 'genre_ids', 'id_x', 'original_language',
       'original_title_x', 'popularity', 'release_date_x', 'title_x',
       'vote_average', 'vote_count', 'Mov_name', 'id_y', 'release_date_y',
       'movie', 'production_budget', 'domestic_gross', 'worldwide_gross',
       'tconst', 'primary_title', 'original_title_y', 'start_year',
       'runtime_minutes', 'genres', 'title_id', 'ordering', 'title_y',
       'region', 'language', 'types', 'attributes', 'is_original_title',
       'domestic_gross_$', 'worldwide_gross_$', 'production_$', 'total_gross',
       'net_cost'],
      dtype='object')

In [28]:
#p_cost[['movie','original_title_x','Mov_name','primary_title','title_y']]
p_cost[['original_title_x','title_x', 'Mov_name', 'movie', 'primary_title','original_title_y','title_y']]

,original_title_x,title_x,Mov_name,movie,primary_title,original_title_y,title_y
0,How to Train Your Dragon,How to Train Your Dragon,howtotrainyourdragon,How to Train Your Dragon,How to Train Your Dragon,How to Train Your Dragon,Como Treinar o Seu Dragão
1,How to Train Your Dragon,How to Train Your Dragon,howtotrainyourdragon,How to Train Your Dragon,How to Train Your Dragon,How to Train Your Dragon,Cómo entrenar a tu dragón
2,How to Train Your Dragon,How to Train Your Dragon,howtotrainyourdragon,How to Train Your Dragon,How to Train Your Dragon,How to Train Your Dragon,Πώς να εκπαιδεύσετε τον δράκο σας
3,How to Train Your Dragon,How to Train Your Dragon,howtotrainyourdragon,How to Train Your Dragon,How to Train Your Dragon,How to Train Your Dragon,Hoe tem je een draak
4,How to Train Your Dragon,How to Train Your Dragon,howtotrainyourdragon,How to Train Your Dragon,How to Train Your Dragon,How to Train Your Dragon,Sådan træner du din drage
...,...,...,...,...,...,...,...
56904,The Box,The Box,thebox,The Box,The Box,The Box,The Box
56905,The Box,The Box,thebox,The Box,The Box,The Box,The Box
56906,The Box,The Box,thebox,The Box,The Box,The Box,The Box
56907,Enough,Enough,enough,Enough,Enough,Enough,Enough


In [30]:
#new df with only relevant info and sorted by the total gross
p_cost_simplified = p_cost[['movie','genres','production_budget','domestic_gross','worldwide_gross','total_gross','net_cost']]
display(p_cost_simplified)
p_cost_simplified.sort_values(by= 'total_gross', ascending = False)

,movie,genres,production_budget,domestic_gross,worldwide_gross,total_gross,net_cost
0,How to Train Your Dragon,"Action,Adventure,Animation","$165,000,000","$217,581,232","$494,870,992",712452224,547452224
1,How to Train Your Dragon,"Action,Adventure,Animation","$165,000,000","$217,581,232","$494,870,992",712452224,547452224
2,How to Train Your Dragon,"Action,Adventure,Animation","$165,000,000","$217,581,232","$494,870,992",712452224,547452224
3,How to Train Your Dragon,"Action,Adventure,Animation","$165,000,000","$217,581,232","$494,870,992",712452224,547452224
4,How to Train Your Dragon,"Action,Adventure,Animation","$165,000,000","$217,581,232","$494,870,992",712452224,547452224
...,...,...,...,...,...,...,...
56904,The Box,"Drama,War","$25,000,000","$15,051,977","$34,356,760",49408737,24408737
56905,The Box,NaN,"$25,000,000","$15,051,977","$34,356,760",49408737,24408737
56906,The Box,Thriller,"$25,000,000","$15,051,977","$34,356,760",49408737,24408737
56907,Enough,"Drama,History","$38,000,000","$39,177,215","$50,970,660",90147875,52147875


,movie,genres,production_budget,domestic_gross,worldwide_gross,total_gross,net_cost
168,Avatar,Horror,"$425,000,000","$760,507,625","$2,776,345,279",3536852904,3111852904
169,Avatar,Horror,"$425,000,000","$760,507,625","$2,776,345,279",3536852904,3111852904
170,Avatar,Horror,"$425,000,000","$760,507,625","$2,776,345,279",3536852904,3111852904
54001,Avengers: Infinity War,"Action,Adventure,Sci-Fi","$300,000,000","$678,815,482","$2,048,134,200",2726949682,2426949682
54000,Avengers: Infinity War,"Action,Adventure,Sci-Fi","$300,000,000","$678,815,482","$2,048,134,200",2726949682,2426949682
...,...,...,...,...,...,...,...
22837,Eden,"Biography,Drama,Music","$2,300,000",$0,$0,0,-2300000
22836,Eden,"Biography,Drama,Music","$2,300,000",$0,$0,0,-2300000
42494,Amnesiac,"Drama,Horror,Thriller","$3,000,000",$0,$0,0,-3000000
42493,To Write Love On Her Arms,"Biography,Drama,Fantasy","$3,400,000",$0,$0,0,-3400000


In [ ]:
#filtered to see all movies with more than $0 global gross
#p_cost_simp_filter = p_cost_simplified[p_cost_simplified['total_gross']>0]
#p_cost_simp_filter.sort_values(by= 'total_gross', ascending = False).head()

In [ ]:
fig, ax =

In [ ]:
############################################

In [ ]:
#tn_db = tn_mov_df.join(tndb_df, on='id', how = 'left',lsuffix = " combined")
#tn_db.head()


In [ ]:
'''


tndb_df["Mov_name"] = tndb_df['movie'].str.lower().str.replace('[ \'\:\,\.]','').astype(str)
tn_mov_df["Mov_name"] = tn_mov_df['title'].str.lower().str.replace('[ \'\:\,\.]','').astype(str)
akas_df["Mov_name"] = akas_df['title'].str.lower().str.replace('[ \'\:\,\.]','').astype(str)
basics_df['Mov_name'] = basics_df['primary_title'].str.lower().str.replace('[ \'\:\,\.]','').astype(str)
'''


#def simplify_title(df_col):
#    df_col['Mov_name'] = df_col.str.lower().str.replace('[ \'\:\,\.]','').astype(str)
display(akas_df.head(1))
#tn_mov_df.head(1)
basics_df.head(1)

In [ ]:
imdb_db = basics_df.join(akas_df,lsuffix ='title_id',rsuffix = "tconst",how = "outer")
imdb_db

In [ ]:
#display(tndb_df.head(1))
#display(tn_mov_df.head(1))
#akas_df.head(1)

In [ ]:
#tn_db = tn_mov_df.merge(tndb_df, on = 'Mov_name', how = 'left')
#tn_db.head()

In [ ]:
#tn_db.info()

#tn_imdb_db = akas_df.merge(tn_db, on = 'Mov_name', how = 'left')
#tn_imdb_db.info()
#tn_imdb_db.head(10)

In [ ]:
#tn_df['Mov_name'] = tn_df['Mov_name'].str.replace('[ \'\:\,\.]','')

In [ ]:
#tn_info_df = tn_mov_df.join(tndb_df, how = "inner")